## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:C:/Users/trinh/Desktop/Projet/02-tracking/scenario/artifact_db/1', creation_time=1765290240756, experiment_id='1', last_update_time=1765290240756, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1765290132548, experiment_id='0', last_update_time=1765290132548, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/12/09 15:48:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


default artifacts URI: 'file:C:/Users/trinh/Desktop/Projet/02-tracking/scenario/artifact_db/1/d5afd77f88b544d6821ae98ccaed7310/artifacts'
🏃 View run adaptable-eel-634 at: http://127.0.0.1:5000/#/experiments/1/runs/d5afd77f88b544d6821ae98ccaed7310
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [ ]:
mlflow.search_experiments()

### Interacting with the model registry

In [5]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [6]:
client.search_registered_models()

[]

In [7]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/12/09 15:49:25 WARNING mlflow.tracking._model_registry.fluent: Run with id d5afd77f88b544d6821ae98ccaed7310 has no artifacts at artifact path 'models', registering model based on models:/m-31ba215d4db04654bace4382c0ea2667 instead
2025/12/09 15:49:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1765291765728, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1765291765728, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='d5afd77f88b544d6821ae98ccaed7310', run_link='', source='models:/m-31ba215d4db04654bace4382c0ea2667', status='READY', status_message=None, tags={}, user_id='', version='1'>